# ASSIGNMENT 1: CREATING OBJECTS

### Author: Andrea Álvarez Pérez

There are three tab-delimited data files:

1. seed_stock_data.tsv: contains information about seeds in your genebank 
2. gene_information.tsv: contains information about genes 
3. cross_data.tsv: contains information about the crosses you have made

Each file begins with a heading line, followed by lines of data.

## Problem 1: 

"Simulate" planting 7 grams of seeds from each of the records in the seed stock genebank. Then you should update the genebank information to show the new quantity of seeds that remain after a planting. The new state of the genebank
should be printed to a new file, using exactly the same format as the original file seed_stock_data.tsv.

If the amount of seed is reduced to zero or less than zero, then a friendly warning message should appear on the screen. The amount of seed left in the gene bank is, of course, not LESS than zero.

In [1]:
# create objects

class SeedStock
  
  attr_accessor :seed_stockID
  attr_accessor :geneID
  attr_accessor :last_planted
  attr_accessor :storage
  attr_accessor :grams_rem
    
  def initialize (params = {}) # default values

    @seed_stockID = params.fetch(:seed_stockID, "000000")
    @geneID = params.fetch(:geneID, "unknown")
    @last_planted = params.fetch(:last_planted, "unknown")
    @storage = params.fetch(:storage, "unknown")
    @grams_rem = params.fetch(:grams_rem, "0")
    
  end

  # write the header in output file. If the file exists, rewrite it
  File.write("new_stock_file.tsv", ["Seed_Stock", "Mutant_Gene_ID", "Last_Planted", "Storage", "Grams_Remaining"].join("\t"), mode: "w") 
   
  def seed_grams_remain ()
    newvalue = grams_rem.to_i - 7 # convert string into integer and plant 7 seeds
    if newvalue <= 0
      # set value into 0 to avoid having negative numbers
      grams_rem = 0
      puts "WARNING: we have run out of Seed Stock #{seed_stockID}."     
    else
      # set newvalue
      grams_rem = newvalue
    end
    # write results in new_stock_file
    File.open("new_stock_file.tsv", "a") { |f| f.write("\r\n#{seed_stockID}\t#{geneID}\t#{last_planted}\t#{storage}\t#{grams_rem}\t") }
  end
  
end


# =======================================================================================================

# Open files and fill class data

require "csv"

# If you use the open method
tsv_seed = CSV.read("seed_stock_data.tsv", col_sep: "\t")[1 .. -1] # delete the header

for i in [0,1,2,3,4]
  p2 = SeedStock.new(
    :seed_stockID => tsv_seed[i][0],      
    :geneID => tsv_seed[i][1],                   
    :last_planted => tsv_seed[i][2], 
    :storage => tsv_seed[i][3],
    :grams_rem => tsv_seed[i][4]
    )
  puts p2.seed_grams_remain()
end



35
36
38
34
35


[0, 1, 2, 3, 4]

Chi-square is used to test hypotheses about the distribution of observations in different categories. 

- The null hypothesis (Ho) is that the observed frequencies are the same as the expected frequencies (except for chance variation). If the observed and expected frequencies are the same, then χ²= 0. 
- Alternative hypothesis (H1). If the frequencies you observe are different from expected frequencies, the value of χ² goes up. 

The larger the value of χ², the more likely it is that the distributions are significantly different.  

In [ ]:
chisq.test()

NameError: undefined local variable or method `chisq' for main:Object

In [ ]:
class SeedStock # superclass
  
  attr_accessor :seed_stockID
  attr_accessor :geneID
  attr_accessor :last_planted
  attr_accessor :storage
  attr_accessor :grams_rem
  attr_accessor :example
    
  def initialize (params = {}) # default values

    @seed_stockID = params.fetch(:seed_stockID, "000000")
    @geneID = params.fetch(:geneID, "unknown")
    @last_planted = params.fetch(:last_planted, "unknown")
    @storage = params.fetch(:storage, "unknown")
    @grams_rem = params.fetch(:grams_rem, "0")
    
  end
  
  def seed_grams_remain ()
    newvalue = grams_rem.to_i - 7 # convert string into integer and plant 7 seeds
    if newvalue <= 0
      # set value into 0 to avoid having negative numbers
      grams_rem = 0
      puts "WARNING: we have run out of Seed Stock #{seed_stockID}."     
    else
      # set newvalue
      grams_rem = newvalue
    end
    # write results in new_stock_file
    File.open("new_stock_file.tsv", "a") { |f| f.write("\r\n#{seed_stockID}\t#{geneID}\t#{last_planted}\t#{storage}\t#{grams_rem}\t") }
  end
  
  def display gene = 0, name = 0
    
    puts "Gene ID: #{gene}, name: #{name}"
    
  end
  
  def display2 parent1 = "unknown"
    
    str2 = ".com"

    puts "Str1 is #{parent1}"
    puts "Str2 is #{str2}"
    str3 = parent1.concat(str2)

    puts "The appended String object is #{str3}"
    
  end
  
  def chi_square f2_wild = 0, f2_P1 = 0, f2_P2 = 0, f2_P1P2 = 0, gene_name = 0
    
    puts "F2_wild: #{f2_wild}"
    new = f2_wild.to_i - 10
    puts "F2_wild - 10 = #{new}"
    puts "F2_P1: #{f2_P1}"
    puts "F2_P2: #{f2_P2}"
    puts "F2_P1P2: #{f2_P1P2}"
    
  end
  
  def calculate f2_wild = 0, f2_P1 = 0, f2_P2 = 0, f2_P1P2 = 0, parent1 = "unknown", parent2 = "unknown"
    
    # First we calculate the expected frequencies
    # 1. Sum of all individuals
    sum = f2_wild.to_i + f2_P1.to_i + f2_P2.to_i + f2_P1P2.to_i
    # 2. 9:3:3:1 proportion
    f2_wild_E = (sum*9)/16
    f2_P1_E = (sum*3)/16
    f2_P2_E = (sum*3)/16
    f2_P1P2_E = sum/16
    
    #3. Chi-square test: (O-E)^2 / E
    chi2 = ( (((f2_wild.to_i - f2_wild_E)**2) / f2_wild_E) + (((f2_P1.to_i - f2_P1_E)**2) / f2_P1_E) + (((f2_P2.to_i - f2_P2_E)**2) / f2_P2_E) + (((f2_P1P2.to_i - f2_P1P2_E)**2) / f2_P1P2_E) )
    
    #4. Cutoff chi2 value: alfa = 0.05 and df = 4 - 1 (Chi-square distribution table)
    cutoff = 7.815
    
    #5. Compare with hypothesis
    if chi2 > cutoff
      puts "Parent 1 #{parent1} and parent 2 #{parent2} genes are linked"
    else
      puts "Parent 1 #{parent1} and parent 2 #{parent2} genes are NOT linked"
    end
    
  end
  
end 

class Gene < SeedStock
  
  attr_accessor :geneID # geneID lo heredo de SeedStock
  attr_accessor :gene_name
  attr_accessor :mutant_phenotype
  
  def initialize (params = {}) # default values
  
    super(params)
    @geneID = params.fetch(:geneID, "00000")
    @gene_name = params.fetch(:gene_name, "unknown")
    @mutant_phenotype = params.fetch(:mutant_phenotype, "unknown")
    
  end
  
  def display 
    
    super
    
    super geneID, gene_name
    
  end
  
end

class HybridCross < SeedStock
  
  attr_accessor :parent1 #parent1 lo heredo de SeedStock
  attr_accessor :parent2
  attr_accessor :f2_wild
  attr_accessor :f2_P1
  attr_accessor :f2_P2
  attr_accessor :f2_P1P2
  
  def initialize (params = {}) # default values
    
    super(params)
    @parent1 = params.fetch(:parent1, "unknown")
    @parent2 = params.fetch(:parent2, "unknown")
    @f2_wild = params.fetch(:f2_wild, "000")
    @f2_P1 = params.fetch(:f2_P1, "000")
    @f2_P2 = params.fetch(:f2_P2, "000")
    @f2_P1P2 = params.fetch(:f2_P1P2, "000")
       
  end
  
  def display2
    
    super parent1
    
  end
  
  def chi_square 
    
    super f2_wild, f2_P1, f2_P2, f2_P1P2
    
  end
  
  def calculate
    
    super f2_wild, f2_P1, f2_P2, f2_P1P2, parent1, parent2
    
  end

end

# =======================================================================================================

# Open files and fill class data

require "csv"

tsv_seed = CSV.read("seed_stock_data.tsv", col_sep: "\t")[1 .. -1] # delete the header

for i in [0,1,2,3,4]
  p2 = SeedStock.new(
    :seed_stockID => tsv_seed[i][0],      
    :geneID => tsv_seed[i][1],                   
    :last_planted => tsv_seed[i][2], 
    :storage => tsv_seed[i][3],
    :grams_rem => tsv_seed[i][4]
    )
end

tsv_gene = CSV.read("gene_information.tsv", col_sep: "\t")[1 .. -1] # delete the header

for i in [0,1,2]
  p3 = Gene.new(    
    :geneID => tsv_gene[i][0],                   
    :gene_name => tsv_gene[i][1], 
    :mutant_phenotype => tsv_gene[i][2],
    )
  #puts p3.display()
end

tsv_cross = CSV.read("cross_data.tsv", col_sep: "\t")[1 .. -1] # delete the header

for i in [0,1,2,3,4,5]
  p4 = HybridCross.new(    
    :parent1 => tsv_cross[i][0],                   
    :parent2 => tsv_cross[i][1], 
    :f2_wild => tsv_cross[i][2],
    :f2_P1 => tsv_cross[i][3],
    :f2_P2 => tsv_cross[i][4],
    :f2_P1P2 => tsv_cross[i][5],
    )
  #puts p4.display2
  puts p4.calculate
end



Parent 1 A334 and parent 2 A348 genes are NOT linked

Parent 1 A348 and parent 2 B3334 genes are NOT linked

Parent 1 B3334 and parent 2 A51 genes are NOT linked

Parent 1 A51 and parent 2 B52 genes are linked

Parent 1 B52 and parent 2 A334 genes are NOT linked



NoMethodError: undefined method `[]' for nil:NilClass

In [33]:
class SeedStock # superclass
  
  attr_accessor :seed_stockID
  attr_accessor :geneID
  attr_accessor :last_planted
  attr_accessor :storage
  attr_accessor :grams_rem
  attr_accessor :example
    
  def initialize (params = {}) # default values

    @seed_stockID = params.fetch(:seed_stockID, "000000")
    @geneID = params.fetch(:geneID, "unknown")
    @last_planted = params.fetch(:last_planted, "unknown")
    @storage = params.fetch(:storage, "unknown")
    @grams_rem = params.fetch(:grams_rem, "0")
    
  end
  
  def chi_square 
    
    
  
end 


# =======================================================================================================

# Open files and fill class data

require "csv"

tsv_seed = CSV.read("seed_stock_data.tsv", col_sep: "\t")[1 .. -1] # delete the header

for i in [0,1,2,3,4]
  p2 = SeedStock.new(
    :seed_stockID => tsv_seed[i][0],      
    :geneID => tsv_seed[i][1],                   
    :last_planted => tsv_seed[i][2], 
    :storage => tsv_seed[i][3],
    :grams_rem => tsv_seed[i][4]
    )
end



[0, 1, 2, 3, 4]

In [ ]:


tsv_gene = CSV.read("gene_information.tsv", col_sep: "\t")[1 .. -1] # delete the header

for i in [0,1,2]
  p3 = Gene.new(    
    :geneID => tsv_gene[i][0],                   
    :gene_name => tsv_gene[i][1], 
    :mutant_phenotype => tsv_gene[i][2],
    )
end

tsv_cross = CSV.read("cross_data.tsv", col_sep: "\t")[1 .. -1] # delete the header

for i in [0,1,2,3,4,5]
  p4 = HybridCross.new(    
    :parent1 => tsv_cross[i][0],                   
    :parent2 => tsv_cross[i][1], 
    :F2_wild => tsv_cross[i][2],
    :F2_P1 => tsv_cross[i][3],
    :F2_P2 => tsv_cross[i][4],
    :F2_P1P2 => tsv_cross[i][5],
    )
  puts p4.chi_square()
end



In [40]:
class Geeks_1
    
    # method of superclass accepting
    # two parameter
    def display a = 0, b = 0
        puts "Parent class, 1st Argument: #{a}, 2nd Argument: #{b}"
    end
end
 
# derived class Geeks_2
class Geeks_2 < Geeks_1
 
    # subclass method having the same name
    # as superclass
    def display a, b
         
        # calling the superclass method
        # by default it will pass
        # both the arguments
        super
         
        # passing only one argument
        super a
         
        # passing both the argument
        super a, b
        # calling the superclass method
        # by default it will not pass
        # both the arguments
        super()
         
        puts "Hey! This is subclass method"
    end
end
 
# creating object of derived class
obj = Geeks_2.new
 
# calling the method of subclass
obj.display "Sudo_Placement", "GFG"

Parent class, 1st Argument: Sudo_Placement, 2nd Argument: GFG
Parent class, 1st Argument: Sudo_Placement, 2nd Argument: 0
Parent class, 1st Argument: Sudo_Placement, 2nd Argument: GFG
Parent class, 1st Argument: 0, 2nd Argument: 0
Hey! This is subclass method


## Problem 2:

Process the information in cross_data.tsv and determine which genes are genetically-linked. To achieve this, you will have to do a Chi-square test on the F2 cross data. If you discover genes that are linked, this information should be added as a property of each of the genes (they are both linked to each other).

